In [1]:
!pip install wfdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.6 MB/s eta 0:00:00a 0:00:01


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import os
import wfdb

In [3]:
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import layers, regularizers
import torch
import random
from IPython import display
from tqdm import tqdm
from shutil import copyfile

In [4]:
PATH = '/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'

ECG_df = pd.read_csv(os.path.join('/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv'), index_col='ecg_id')
ECG_df.scp_codes = ECG_df.scp_codes.apply(lambda x: ast.literal_eval(x))
ECG_df.patient_id = ECG_df.patient_id.astype(int)
ECG_df.nurse = ECG_df.nurse.astype('Int64')
ECG_df.site = ECG_df.site.astype('Int64')
ECG_df.validated_by = ECG_df.validated_by.astype('Int64')

SCP_df = pd.read_csv(os.path.join('/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/scp_statements.csv'), index_col=0)
SCP_df = SCP_df[SCP_df.diagnostic == 1]

ECG_df

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709,56.0,1,NaN,63.0,2,0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
2,13243,19.0,0,NaN,70.0,2,0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
3,20372,37.0,1,NaN,69.0,2,0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
4,17014,24.0,0,NaN,82.0,2,0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
5,17448,19.0,1,NaN,70.0,2,0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21833,17180,67.0,1,NaN,NaN,1,2,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,...,True,NaN,", alles,",NaN,NaN,1ES,NaN,7,records100/21000/21833_lr,records500/21000/21833_hr
21834,20703,93.0,0,NaN,NaN,1,2,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,...,True,NaN,NaN,NaN,NaN,NaN,NaN,4,records100/21000/21834_lr,records500/21000/21834_hr
21835,19311,59.0,1,NaN,NaN,1,2,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,...,True,NaN,", I-AVR,",NaN,NaN,NaN,NaN,2,records100/21000/21835_lr,records500/21000/21835_hr


In [5]:
def diagnostic_class(scp):
    res = set()
    for k in scp.keys():
        if k in SCP_df.index:
            res.add(SCP_df.loc[k].diagnostic_class)
    return list(res)

def diagnostic_subclass(scp):
    res = set()
    for k in scp.keys():
        if k in SCP_df.index:
            res.add(SCP_df.loc[k].diagnostic_subclass)
    return list(res)

ECG_df['scp_classes'] = ECG_df.scp_codes.apply(diagnostic_class)
ECG_df['scp_subclasses'] = ECG_df.scp_codes.apply(diagnostic_subclass)

In [6]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(os.path.join(path, f)) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(os.path.join(path, f)) for f in df.filename_lr]
        data = np.array([signal for signal, meta in data])
    return data

sampling_rate = 500
ECG_data = load_raw_data(ECG_df, sampling_rate, PATH)
ECG_data.shape

(21837, 1000, 12)

In [7]:
X = pd.DataFrame(index=ECG_df.index)

X['age'] = ECG_df.age
X.age.fillna(0, inplace=True)

X['sex'] = ECG_df.sex.astype(float)
X.sex.fillna(0, inplace=True)

X['height'] = ECG_df.height
X.loc[X.height < 50, 'height'] = np.nan
X.height.fillna(0, inplace=True)

X['weight'] = ECG_df.weight
X.weight.fillna(0, inplace=True)

X['infarction_stadium1'] = ECG_df.infarction_stadium1.replace({
    'unknown': 0,
    'Stadium I': 1,
    'Stadium I-II': 2,
    'Stadium II': 3,
    'Stadium II-III': 4,
    'Stadium III': 5
}).fillna(0)

X['infarction_stadium2'] = ECG_df.infarction_stadium2.replace({
    'unknown': 0,
    'Stadium I': 1,
    'Stadium II': 2,
    'Stadium III': 3
}).fillna(0)

X['pacemaker'] = (ECG_df.pacemaker == 'ja, pacemaker').astype(float)

X

,age,sex,height,weight,infarction_stadium1,infarction_stadium2,pacemaker
ecg_id,,,,,,,
1,56.0,1.0,0.0,63.0,0.0,0.0,0.0
2,19.0,0.0,0.0,70.0,0.0,0.0,0.0
3,37.0,1.0,0.0,69.0,0.0,0.0,0.0
4,24.0,0.0,0.0,82.0,0.0,0.0,0.0
5,19.0,1.0,0.0,70.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
21833,67.0,1.0,0.0,0.0,0.0,0.0,0.0
21834,93.0,0.0,0.0,0.0,4.0,0.0,0.0
21835,59.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
Z = pd.DataFrame(0, index=ECG_df.index, columns=['NORM', 'MI', 'STTC', 'CD', 'HYP'], dtype='int')
for i in Z.index:
    for k in ECG_df.loc[i].scp_classes:
        Z.loc[i, k] = 1

Z

,NORM,MI,STTC,CD,HYP
ecg_id,,,,,
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
...,...,...,...,...,...
21833,0,0,1,0,0
21834,1,0,0,0,0
21835,0,0,1,0,0


In [25]:
X_train, Y_train, Z_train = X[ECG_df.strat_fold <= 8],  ECG_data[X[ECG_df.strat_fold <= 8].index - 10919],  Z[ECG_df.strat_fold <= 8]
X_valid, Y_valid, Z_valid = X[ECG_df.strat_fold == 9],  ECG_data[X[ECG_df.strat_fold == 9].index - 10919],  Z[ECG_df.strat_fold == 9]
X_test,  Y_test,  Z_test  = X[ECG_df.strat_fold == 10], ECG_data[X[ECG_df.strat_fold == 10].index - 10919], Z[ECG_df.strat_fold == 10]

print(X_train.shape, Y_train.shape, Z_train.shape)
print(X_valid.shape, Y_valid.shape, Z_valid.shape)
print(X_test.shape,  Y_test.shape,  Z_test.shape)

(17441, 7) (17441, 1000, 12) (17441, 5)
(2193, 7) (2193, 1000, 12) (2193, 5)
(2203, 7) (2203, 1000, 12) (2203, 5)


In [26]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler

class RNNDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.tensor(X, dtype=torch.float32)
        self.labels = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.data[idx], self.labels[idx]
        else:
            return self.data[idx]

mask_train = (Z_train['NORM'] == 1) | (Z_train['MI'] == 1)
X_train_filtered = X_train[mask_train]
Y_train_filtered = Y_train[mask_train]
Z_train_filtered = Z_train[mask_train].drop(columns=['STTC', 'CD', 'HYP'])

mask_valid = (Z_valid['NORM'] == 1) | (Z_valid['MI'] == 1)
X_valid_filtered = X_valid[mask_valid]
Y_valid_filtered = Y_valid[mask_valid]
Z_valid_filtered = Z_valid[mask_valid].drop(columns=['STTC', 'CD', 'HYP'])

mask_test = (Z_test['NORM'] == 1) | (Z_test['MI'] == 1)
X_test_filtered = X_test[mask_test]
Y_test_filtered = Y_test[mask_test]
Z_test_filtered = Z_test[mask_test].drop(columns=['STTC', 'CD', 'HYP'])

X_scaler = MinMaxScaler()
X_scaler.fit(X_train_filtered)
X_train_filtered_scaled = pd.DataFrame(X_scaler.transform(X_train_filtered), columns=X_train_filtered.columns)
X_valid_filtered_scaled = pd.DataFrame(X_scaler.transform(X_valid_filtered), columns=X_valid_filtered.columns)
X_test_filtered_scaled  = pd.DataFrame(X_scaler.transform(X_test_filtered), columns=X_test_filtered.columns)

Y_scaler = MinMaxScaler()
Y_scaler.fit(Y_train_filtered.reshape(-1, Y_train_filtered.shape[-1]))
Y_train_filtered_scaled = Y_scaler.transform(Y_train_filtered.reshape(-1, Y_train_filtered.shape[-1])).reshape(Y_train_filtered.shape)
Y_valid_filtered_scaled = Y_scaler.transform(Y_valid_filtered.reshape(-1, Y_valid_filtered.shape[-1])).reshape(Y_valid_filtered.shape)
Y_test_filtered_scaled  = Y_scaler.transform(Y_test_filtered.reshape(-1, Y_test_filtered.shape[-1])).reshape(Y_test_filtered.shape)

X_train = Y_train_filtered_scaled  
X_valid = Y_valid_filtered_scaled
X_test  = Y_test_filtered_scaled

y_train = Z_train_filtered[['NORM', 'MI']].values
y_valid = Z_valid_filtered[['NORM', 'MI']].values
y_test  = Z_test_filtered[['NORM', 'MI']].values

train_dataset = RNNDataset(X_train, y_train)
valid_dataset = RNNDataset(X_valid, y_valid)
test_dataset = RNNDataset(X_test, y_test)

from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print("Train loader size:", len(train_loader.dataset))
print("Validation loader size:", len(valid_loader.dataset))
print("Test loader size:", len(test_loader.dataset))

Train loader size: 11995
Validation loader size: 1501
Test loader size: 1517


In [73]:
class ECGRNN(nn.Module):
    def __init__(self, rnn_type, input_size, hidden_size, num_layers, output_size=12):
        super(ECGRNN, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        if rnn_type == 'RNN':
            self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, 
                              dropout=(0 if num_layers == 1 else 0.05), 
                              num_layers=num_layers, batch_first=True, bidirectional=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, 
                              dropout=(0 if num_layers == 1 else 0.05), 
                              num_layers=num_layers, batch_first=True, bidirectional=True)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, 
                               dropout=(0 if num_layers == 1 else 0.05), 
                               num_layers=num_layers, batch_first=True, bidirectional=True)
        else:
            raise(ValueError('Incorrect choice of RNN supplied'))

        self.batch_norm = nn.BatchNorm1d(hidden_size * 2)  

        self.fc1 = nn.Linear(hidden_size * 2, 128) 
        self.fc2 = nn.Linear(128, output_size)  
        
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, h_state=None):
        batch_size = x.size(0)
        
        if h_state is None:
            if isinstance(self.rnn, nn.LSTM):
                h_state = (torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(x.device),
                           torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(x.device))
            else:
                h_state = torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(x.device)
        
        r_out, h_state = self.rnn(x, h_state)

        r_out_last = r_out[:, -1, :]  # Shape: (batch_size, hidden_size * 2)

        r_out_last = self.batch_norm(r_out_last)
        r_out_last = self.dropout(r_out_last)
        
        fc1_out = torch.relu(self.fc1(r_out_last))
        final_y = self.fc2(fc1_out)

        return final_y, h_state

In [150]:
import tensorflow as tf
import numpy as np

class MMDStatistic:
    def __init__(self, n_samples_target, n_samples_pred):
        self.n_samples_target = n_samples_target
        self.n_samples_pred = n_samples_pred

    def __call__(self, sample_target, sample_pred, bandwidths):
        target_kernel = tf.matmul(sample_target, sample_target, transpose_b=True)
        pred_kernel = tf.matmul(sample_pred, sample_pred, transpose_b=True)
        cross_kernel = tf.matmul(sample_target, sample_pred, transpose_b=True)
        
        return tf.reduce_mean(target_kernel + pred_kernel - 2 * cross_kernel)

def mmd(targets, predictions):
    targets = tf.reshape(targets, (-1, targets.shape[-1]))  
    predictions = tf.reshape(predictions, (-1, predictions.shape[-1]))  
    
    mmd_stat = MMDStatistic(targets.shape[0], predictions.shape[0])
    
    stat = mmd_stat(targets, predictions, [1.])  # Параметр bandwidth
    return stat.numpy()

def rmse(targets, predictions):
    targets = tf.cast(targets, dtype=tf.float64)
    predictions = tf.cast(predictions, dtype=tf.float64)
    
    if targets.shape != predictions.shape:
        raise ValueError(f"Shape mismatch: targets {targets.shape}, predictions {predictions.shape}")
    
    return np.sqrt(np.mean((targets - predictions) ** 2))

def prd(targets, predictions):
    targets = tf.cast(targets, dtype=tf.float32)
    predictions = tf.cast(predictions, dtype=tf.float32)
    
    if len(targets.shape) == 3:  # Если это батч RNN данных
        targets = tf.reshape(targets, (-1, targets.shape[-1]))  # Плоский вид
        predictions = tf.reshape(predictions, (-1, predictions.shape[-1]))  # Плоский вид
    
    if targets.shape != predictions.shape:
        raise ValueError(f"Shape mismatch: targets {targets.shape}, predictions {predictions.shape}")
    
    epsilon = 1e-6
    numerator = tf.sqrt(tf.reduce_mean((targets - predictions) ** 2))
    denominator = tf.sqrt(tf.reduce_mean(targets ** 2)) + epsilon
    return (numerator / denominator).numpy()

In [151]:
def generate_predictions(model, dataloader, init_sequence_length, mmd_stat_func, rmse_func, prd_func):
    model.eval()
    final_outputs = []
    mmd_values = []
    rmse_values = []
    prd_values = []

    with torch.no_grad():
            initial_input = torch.zeros((1, init_sequence_length, 12)).to(device)  # (batch_size=1, seq_length, features)
            h_state = torch.zeros(model.num_layers * 2, 1, model.hidden_size).to(device)  # (num_layers * 2 для bidirectional)
        
            prediction_length = len(dataloader.dataset.labels) - init_sequence_length
        
            for i in range(prediction_length):
                output, h_state = model(initial_input, h_state)
    
                predicted_features = output.unsqueeze(1)  
                predicted_features = predicted_features.to(device)  
        
                zeros = torch.zeros(predicted_features.size(0), predicted_features.size(1), 10).to(device)  
                predicted_features = torch.cat([predicted_features, zeros], dim=2)  

            initial_input = torch.cat((initial_input[:, 1:, :], predicted_features), dim=1)
            
            final_outputs.append(output.cpu().squeeze().numpy())  

            target = dataloader.dataset.labels[i + init_sequence_length]
            
            mmd_value = mmd_stat_func(target, output.cpu().squeeze().numpy())
            mmd_values.append(mmd_value)
            
            rmse_value = rmse_func(target, output.cpu().squeeze().numpy())
            rmse_values.append(rmse_value)
            
            prd_value = prd_func(target, output.cpu().squeeze().numpy())
            prd_values.append(prd_value)

    print('MMD:', f'mean={np.mean(mmd_values):.6f}', f'min={np.min(mmd_values):.6f}', f'max={np.max(mmd_values):.6f}')
    print('PRD:', f'mean={np.mean(prd_values):.4f}', f'min={np.min(prd_values):.4f}', f'max={np.max(prd_values):.4f}')
    print('RMSE:', f'mean={np.mean(rmse_values):.4f}', f'min={np.min(rmse_values):.4f}', f'max={np.max(rmse_values):.4f}')
    
    return np.array(final_outputs), np.array(mmd_values), np.array(rmse_values), np.array(prd_values)



def plot_predictions(final_outputs, actual_labels, init_sequence_length, output_file='sin_wave.png'):
    for i in range(12):
        plt.figure(figsize=(10, 6))
        plt.plot(final_outputs[:, i], label=f'Predicted Channel {i+1}')
        plt.plot(actual_labels[init_sequence_length:, i], label=f'Actual Channel {i+1}')
        plt.legend(loc='upper right')
        plt.xlabel('Time steps')
        plt.ylabel('Values')
        plt.title(f'Model Predictions vs Actual (Channel {i+1})')
        plt.grid(True)
        plt.tight_layout()

        plt.savefig(f'{output_file}_channel_{i+1}.png')
        plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

LEARNING_RATE = 0.09
BATCH_SIZE = 16
NUM_EPOCHS = 50
SEQUENCE_LENGTH = 1000
RNN_TYPE = 'RNN'  # Или 'RNN', 'LSTM'

def train_model(model, dataloader, loss_function, optimizer, epochs):
    model.train()
    loss_all = []

    for epoch in range(epochs):
        epoch_loss = 0  
        epoch_mse = 0  
        epoch_mae = 0  
        epoch_cc = 0 
        correct = 0  
        total = 0  

        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            batch_size = x_batch.size(0)

            if isinstance(model.rnn, nn.LSTM):
                h_state = (torch.zeros(model.num_layers * 2, batch_size, model.hidden_size).to(device),
                           torch.zeros(model.num_layers * 2, batch_size, model.hidden_size).to(device))
            else:
                h_state = torch.zeros(model.num_layers * 2, batch_size, model.hidden_size).to(device)

            optimizer.zero_grad()

            output, _ = model(x_batch, h_state)

            loss = loss_function(output, y_batch)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()  
            epoch_mse += ((output - y_batch) ** 2).mean().item()  
            epoch_mae += torch.abs(output - y_batch).mean().item()  
            epoch_cc += np.corrcoef(output.cpu().detach().numpy().flatten(), y_batch.cpu().detach().numpy().flatten())[0, 1]  # CC

            if output.size(1) > 1:  
                correct += (output.argmax(dim=1) == y_batch.argmax(dim=1)).sum().item()
                total += y_batch.size(0)

        avg_epoch_loss = epoch_loss / len(dataloader)
        avg_epoch_mse = epoch_mse / len(dataloader)
        avg_epoch_mae = epoch_mae / len(dataloader)
        avg_epoch_cc = epoch_cc / len(dataloader)

        if total > 0:
            avg_epoch_accuracy = correct / total
        else:
            avg_epoch_accuracy = 0

        print(f'Epoch {epoch+1}/{epochs} - '
              f'Loss: {avg_epoch_loss:.4f}, '
              f'MSE: {avg_epoch_mse:.4f}, '
              f'MAE: {avg_epoch_mae:.4f}, '
              f'CC: {avg_epoch_cc:.4f}, '
              f'Accuracy: {avg_epoch_accuracy:.4f}')
        
        loss_all.append(avg_epoch_loss)

    return loss_all


    torch.save(model.state_dict(), 'trained_ecg_model.pt')

rnn = ECGRNN(RNN_TYPE, input_size=12, hidden_size=4, num_layers=2, output_size=2).to(device)
optimizer = torch.optim.AdamW(rnn.parameters(), lr=LEARNING_RATE)
loss_function = nn.BCEWithLogitsLoss()

loss_all = train_model(rnn, dataloader=train_loader, loss_function=loss_function, optimizer=optimizer, epochs=NUM_EPOCHS)

generate_predictions(rnn, test_loader, SEQUENCE_LENGTH, mmd_stat_func=mmd, rmse_func=rmse, prd_func=prd)
final_outputs, mmd_values, rmse_values, prd_values = generate_predictions(rnn, test_loader, SEQUENCE_LENGTH, mmd_stat_func=mmd, rmse_func=rmse, prd_func=prd)


print(f"MMD Mean: {np.mean(mmd_values)}")
print(f"RMSE Mean: {np.mean(rmse_values)}")
print(f"PRD Mean: {np.mean(prd_values)}")

plot_predictions(final_outputs, test_loader.dataset.labels, init_sequence_length=SEQUENCE_LENGTH)